# Implementing a Feed-Foward Network in PyTorch

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import PyTorch
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Load the data 
data = np.loadtxt(open("../data/Absenteeism_at_work.csv", "rb"), delimiter=";", skiprows=1)
# Use all data as input with the exception of one column (and convert to float)
input_ = torch.from_numpy(np.delete(data, 14, axis=1)).float()
# Use the column is_social_drinker? as target for prediction (set it to the right dimention and float)
# is_social_drinker is easier to train than is_social_smoker (15) as the data are more balanced. 
target = torch.from_numpy(data[:, 14].reshape(data.shape[0], 1)).float()

# Create training and test datasets with 500 and 240 elements respectively
subsets = torch.utils.data.random_split(target, [500, 240])
training_input = input_[subsets[0].indices]
training_target = target[subsets[0].indices]
test_input = input_[subsets[1].indices]
test_target = target[subsets[1].indices]

In [ ]:
# see https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py
class Net(nn.Module):
    
    # Constructor
    # All elements of `self` are fields of a new object. 
    def __init__(self):
        super(Net, self).__init__()
        # We define a net with three linear hidden layers
        self.fc1 = nn.Linear(20, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 1)

    # PyTorch is clever enough to automatically generate `backward()`
    def forward(self, x):
        # with ReLU activation functions
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # Sigmoid as activation in the last layer
        x = torch.sigmoid(self.fc3(x))
        return x

In [ ]:
# Create a new net with randomly initialised parameters (weight matrices)
net = Net()
# The parameters (weight matrices) are randomly initialised
#list(net.parameters())

In [ ]:
# Training function
def training(learning_rate) :
    # Calculate the output based on current parameters
    prediction = net(training_input)
    # Calculate loss
    loss = F.binary_cross_entropy(prediction, training_target)
    # Back-propagate
    net.zero_grad()
    loss.backward()
    # Update the weight matrices
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)
    return loss

def accuracy(input_, target):
    prediction = net(input_)
    result = (prediction > 0.5) == target.type(torch.ByteTensor)
    return sum(result.type(torch.FloatTensor)) / len(result)

In [ ]:
# The untrained network generates random predictions and its accuracy is about 50% for balanced dataset,
# or (1 - ) sum(target) / len(target), respectively.
accuracy(input_, target)

In [ ]:
# Learning rate must be < 1, small enough for the parameters to converge and large enough for an efficient training.
# Vary this value to see how the training evolves.
learning_rate = 0.001 
# Number of iterations.
n_iterations = 10000
for i in range(0, n_iterations):
    result = training(learning_rate)
print(result)

In [ ]:
accuracy(training_input, training_target)

In [ ]:
accuracy(test_input, test_target)